Вашим заданием в этом модуле будет проанализировать и преобразовать данные о велопоездках клиентов компании Citi Bike (США), специализирующейся на прокате велосипедов.

Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

starttime — время начала поездки (дата, время);

stoptime — время окончания поездки (дата, время);

start station id — идентификатор стартовой стоянки;

start station name — название стартовой стоянки;

start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;

end station id — идентификатор конечной стоянки;

end station name — название конечной стоянки;

end station latitude, end station longitude — географическая широта и долгота конечной стоянки;

bikeid — идентификатор велосипеда;

usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);

birth year — год рождения клиента;

gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).

In [1]:
import pandas as pd

In [2]:
csv = pd.read_csv('data\citibike-tripdata.csv')

In [138]:
df = csv.copy()

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

In [140]:
# Сколько пропусков в столбце start station id?
df[df['start station id'].isnull()].shape[0]

169

In [141]:
# Какой тип данных имеют столбцы starttime и stoptime?
df.dtypes['starttime']

dtype('O')

In [142]:
# Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа
df['start station id'].value_counts().index[0]

281.0

In [143]:
# Велосипед с каким идентификатором является самым популярным?
df['bikeid'].value_counts().index[0]

33887

In [144]:
# Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. 
# Ответ округлите до сотых.
df['usertype'].value_counts(normalize=True)

Subscriber    0.774007
Customer      0.225993
Name: usertype, dtype: float64

In [145]:
# Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше
df['gender'].value_counts()

1    183582
2     74506
0     41912
Name: gender, dtype: int64

In [146]:
# Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты, не равны друг другу
start_nuique = df['start station id'].nunique()
end_nuique = df['end station id'].nunique()
start_nuique != end_nuique

True

In [147]:
# В рассматриваемые дни минимальный возраст клиента составлял 10 лет
ages = 2018 - df['birth year']
ages.min() == 10

False

In [148]:
# Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, 
# является стоянка с названием "Eastern Pkwy & Washington Ave"
df['start station name'].value_counts().index[-1] == "Eastern Pkwy & Washington Ave"


True

In [149]:
# Наибольшее количество поездок завершается на стоянке под названием "Liberty Light Rail"
df['end station name'].value_counts().index[0] == "Liberty Light Rail"

False

In [150]:
# В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: 
# это столбцы с идентификатором и названием стартовой и конечной стоянки. 
# Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?
df.drop(['start station id', 'end station id'], axis=1, inplace=True)

In [151]:
df.shape[1]

12

In [152]:
# Замените признак birth year на более понятный признак возраста клиента age. 
# Годом отсчёта возраста выберите 2018 год. Столбец birth year удалите из таблицы. 
# Сколько поездок совершено клиентами старше 60 лет?
df['age'] = 2018 - df['birth year']
df['age'] = df['age'].astype('int')
df.drop('birth year', axis=1, inplace=True)

In [153]:
df[df['age'] > 60].shape[0]

11837

In [154]:
# Создайте признак длительности поездки trip duration. 
# Для этого вычислите интервал времени между временем окончания поездки (stoptime) 
# и временем её начала (starttime) в секундах. 
# В качестве ответа запишите среднюю длительность поездки в секундах. 
# Ответ округлите до целого.
df['starttime'] = pd.to_datetime(df['starttime'])
df['stoptime'] = pd.to_datetime(df['stoptime'])

In [155]:
df['trip duration'] = (df['stoptime'] - df['starttime']).dt.seconds

In [156]:
round(df['trip duration'].mean(), 0)

1000.0

In [157]:
# Создайте «признак-мигалку» weekend, который равен 1, 
# если поездка начиналась в выходной день (суббота или воскресенье), 
# и 0 — в противном случае. 
# Выясните, сколько поездок начиналось в выходные.

In [158]:
df['weekend'] = df['starttime'].apply(lambda x: 1 if x.dayofweek == 5 or x.dayofweek == 6 else 0)

In [159]:
df['weekend'].value_counts()

0    184865
1    115135
Name: weekend, dtype: int64

In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   starttime                300000 non-null  datetime64[ns]
 1   stoptime                 300000 non-null  datetime64[ns]
 2   start station name       299831 non-null  object        
 3   start station latitude   300000 non-null  float64       
 4   start station longitude  300000 non-null  float64       
 5   end station name         299831 non-null  object        
 6   end station latitude     300000 non-null  float64       
 7   end station longitude    300000 non-null  float64       
 8   bikeid                   300000 non-null  int64         
 9   usertype                 300000 non-null  object        
 10  gender                   300000 non-null  int64         
 11  age                      300000 non-null  int32         
 12  trip duration   

In [161]:
# Создайте признак времени суток поездки time_of_day. 
# Время суток будем определять из часа начала поездки. 
# Условимся, что:
# поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
# поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
# поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
# поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).

In [162]:
def get_time_of_day(start_datetime):
    time_of_day = [('night', range(7)), ('morning',range(7, 13)), ('day', range(13, 19)), ('evening', range(19, 25))]
    
    current_hour = start_datetime.hour
    
    if current_hour in time_of_day[0][1]:
        return time_of_day[0][0]
    elif current_hour in time_of_day[1][1]:
        return time_of_day[1][0]
    elif current_hour in time_of_day[2][1]:
        return time_of_day[2][0]
    elif current_hour in time_of_day[3][1]:
        return time_of_day[3][0]
    
    return None

In [163]:
df['time_of_day'] = df['starttime'].apply(get_time_of_day)

In [164]:
df['time_of_day'] = df['time_of_day'].astype('category')

In [165]:
df['time_of_day'].cat.codes

0         3
1         3
2         3
3         3
4         3
         ..
299995    1
299996    1
299997    1
299998    1
299999    1
Length: 300000, dtype: int8

In [167]:
df['time_of_day'].cat.categories

Index(['day', 'evening', 'morning', 'night'], dtype='object')

In [172]:
# Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, 
# совёршенных ночью, за представленный в данных период времени? Ответ округлите до целых.
day_ride = df[df['time_of_day'] == 'day'].shape[0]
night_ride = df[df['time_of_day'] == 'night'].shape[0]

In [177]:
round(day_ride / night_ride, 0)

9.0

In [175]:
counts = df['time_of_day'].value_counts()
counts

day        143012
morning     95530
evening     46373
night       15085
Name: time_of_day, dtype: int64

In [178]:
round(counts['day'] / counts['night'], 0)

9.0

In [179]:
def get_columns_unique_info_df(df):
    # создаём пустой список
    unique_list = []
    # пробегаемся по именам столбцов в таблице
    for col in df.columns:
        # создаём кортеж (имя столбца, число уникальных значений)
        item = (col, df[col].nunique(),df[col].dtype) 
        # добавляем кортеж в список
        unique_list.append(item) 
    # создаём вспомогательную таблицу и сортируем её
    unique_counts = pd.DataFrame(
        unique_list,
        columns=['Column Name', 'Num Unique', 'Type']
    ).sort_values(by='Num Unique',  ignore_index=True)
    
    return unique_counts

In [180]:
get_columns_unique_info_df(df)

,Column Name,Num Unique,Type
0,usertype,2,object
1,weekend,2,int64
2,gender,3,int64
3,time_of_day,4,category
4,age,91,int32
5,start station name,759,object
6,end station name,765,object
7,start station latitude,770,float64
8,start station longitude,772,float64
9,end station latitude,775,float64
